<a href="https://colab.research.google.com/github/mykolesiko/egg_article/blob/main/MADE_%D0%A1NN2_leak_beta1_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Статья Multimodal Physiological Signal Emotion Recognition Based on Convolutional Recurrent Neural Network
https://drive.google.com/file/d/1HrkFFrHnpqEs01KkcVgMvkpx7IaHq-C5/view?usp=sharing

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset,  WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

In [ ]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/deap")

In [ ]:
def get_padding(in_size, kernel_size, stride):
    if (in_size % stride == 0):
        padding = max(kernel_size - stride, 0)
    else:
        padding = max(kernel_size - (in_size % stride), 0)
    return (padding)    

In [ ]:
def get_feature_extractor():
  conv1 = nn.Conv2d(1, 64, kernel_size = (3, 3), stride=(1, 1), padding='same')
  relu1 = nn.ReLU()
  bn1 = nn.BatchNorm2d(64)
  conv2 = nn.Conv2d(64, 128, kernel_size = (3, 3), stride=(1, 1), padding='same')
  relu2 = nn.ReLU()
  bn2 = nn.BatchNorm2d(128)
  conv3 = nn.Conv2d(128, 256, kernel_size = (3, 3), stride=(1, 1), padding='same')
  relu3 = nn.ReLU()
  bn3 = nn.BatchNorm2d(256)
  #result = torch.nn.Sequential(conv1, bn1, relu1, conv2, bn2, relu2, conv3, bn3, relu3)
  result = torch.nn.Sequential(conv1, bn1, conv2, bn2, conv3, bn3)
  return(result)

In [ ]:
import glob
import pickle
from sklearn.preprocessing import MinMaxScaler

data = []
labels = []
data_dir = './data_preprocessed_python'
files = glob.glob(os.path.join(data_dir, "*.dat"))
data_raw = []
for file_data in files:
    raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
    data.append(raw_data['data'])
    #data_raw.append(raw_data['data'][:, :, :])
    # data_raw[-1][0, :32, 0]
    # print(data_raw[-1][:, :31, :].min())
    # print(data_raw[-1][:, :31, :].max())
    # scaler = MinMaxScaler()
    # for i in range(40):
    #     #scaler.fit(data[-1][i])
    #     scaler = MinMaxScaler()
    #     data[-1][i] = scaler.fit_transform(data[-1][i])
    #     print(data_raw[-1][i])
    #     print(data[-1][i])
    #     break
    labels.append(raw_data['labels'])
    #break


In [ ]:
NSUBJECTS = 32
NVIDEOS = 40
NCANALS = 32
def remove_baseline():
  for sub in range(NSUBJECTS):
    for nvideo in range(NVIDEOS):
        for ncanal in range(NCANALS):
            baseline_mean = (data[sub][nvideo, ncanal, 0: 128] + data[sub][nvideo, ncanal, 128: 128  *2] + data[sub][nvideo, ncanal, 128 * 2: 128 * 3])/3
            #print(baseline_mean.shape)
            for nsec in range(60):
                     data[sub][nvideo, ncanal, nsec * 128 : nsec * 128 + 128] = data[sub][nvideo, ncanal, nsec * 128 : nsec * 128 + 128] - baseline_mean

remove_baseline()

In [ ]:
class EmotionNet(torch.nn.Module): 
   def __init__(self, hcanals, wcanals, nfeatures, ntimes_in_sample):
      super().__init__()
      #print("1")
      self.convs = nn.ModuleList([get_feature_extractor() for _ in range(ntimes_in_sample)])
      self.conv2 = nn.Conv2d(256 * ntimes_in_sample, 15, kernel_size = 1, stride=1)
      self.relu = nn.ReLU()
      self.flat = nn.Flatten(1, 3)
      self.fc = nn.Linear(1215, 2)
      
   def forward(self, input):
      #print(f"input_shape = {input.shape}")
      input = input.permute(0, 3, 1, 2)
      #input (bs, time = 128,  h=9, w=9)

      outputs = []
      for i, conv in enumerate(self.convs):
            outputs.append(conv(input[:, i, :, :].unsqueeze(1)))
            #print(outputs[i])

      output = torch.cat(outputs, dim = 1)
      #output (bs, s * 256,  h=9, w=9)
      output1 = self.conv2(output)
      #output1 = self.relu(output1)


      output2 = output1.permute(0, 2, 3, 1)
      output3 = self.flat(output2)
      output3 = self.fc(output3)
      #output1 (bs, 15,  h=9, w=9)
      return output3

In [ ]:
LEN_RECORD_IN_SECONDS = 60
NVIDEOS = 40
HCANALS = 9
WCANALS = 9
NTIMES_IN_SAMPLE = 128
NTIMES_IN_SEC = 128
NCANALS = 32
NFEATURES = 32
electrode_matrix = {}
electrode_matrix['FP1'] = [0, 3]
electrode_matrix['FP2'] = [0, 5]
electrode_matrix['AF3'] = [1, 3]
electrode_matrix['AF4'] = [1, 5]
electrode_matrix['F7']  = [2, 0]
electrode_matrix['F3']  = [2, 2]
electrode_matrix['FZ']  = [2, 4]
electrode_matrix['F4']  = [2, 6]
electrode_matrix['F8']  = [2, 8]
electrode_matrix['FC5']  = [3, 1]
electrode_matrix['FC1']  = [3, 3]
electrode_matrix['FC2']  = [3, 5]
electrode_matrix['FC6']  = [3, 7]
electrode_matrix['T7']  = [4, 0]
electrode_matrix['C3']  = [4, 2]
electrode_matrix['CZ']  = [4, 4]
electrode_matrix['C4']  = [4, 6]
electrode_matrix['T8']  = [4, 8]
electrode_matrix['CP5']  = [5, 1]
electrode_matrix['CP1']  = [5, 3]
electrode_matrix['CP2']  = [5, 5]
electrode_matrix['CP6']  = [5, 7]
electrode_matrix['P7']  = [6, 0]
electrode_matrix['P3']  = [6, 2]
electrode_matrix['PZ']  = [6, 4]
electrode_matrix['P4']  = [6, 6]
electrode_matrix['P8']  = [6, 8]
electrode_matrix['PO3'] = [7, 3]
electrode_matrix['PO4'] = [7, 5]
electrode_matrix['O1'] = [8, 3]
electrode_matrix['OZ'] = [8, 4]
electrode_matrix['O2'] = [8, 5]

list_electrodes = ['FP1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3',	'T7',	'CP5',	'CP1',	'P3',	'P7',	'PO3',	'O1',	'OZ',	'PZ',	'FP2',	'AF4', 'FZ', 'F4', 'F8', 'FC6',	'FC2',	'CZ', 'C4', 'T8', 'CP6',	'CP2',	'P4', 	'P8',	'PO4',	'O2']
data_dir = './data_preprocessed_python'
TRAIN_SIZE = 0.9
THRESHOLD = 5

In [ ]:
import glob
import pickle
from collections import Counter

class EmotionDataset(Dataset):
    def __init__ (self, data_dir, type, ind, data, labels):
       self.data = []
       self.labels = []
       self.cnt = [Counter(), Counter(), Counter(),Counter()]
       #data_dir = './data_preprocessed_python'
       #files = glob.glob(os.path.join(data_dir, "*.dat"))[0:1]
       self.type = type
       #split = int(LEN_RECORD_IN_SECONDS)# *  TRAIN_SIZE)
       self.ind = ind
       self.len_files = []
       for s in range(len(data)):
            #print(file_data)
            #raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
            #print(raw_data['data'].shape)
            #labels = raw_data['labels']
            self.data.append(data[s][ind, :, 3 * NTIMES_IN_SEC :LEN_RECORD_IN_SECONDS * NTIMES_IN_SEC + 3 * NTIMES_IN_SEC])
            self.len_files.append(len(ind) *  LEN_RECORD_IN_SECONDS - 1)
            self.len_record = LEN_RECORD_IN_SECONDS
            labels_bin_sub = (labels[s] >= THRESHOLD)
            self.labels.append(labels_bin_sub[ind])          
            for i in range(4):
              self.cnt[i].update(list(self.labels[-1][:, i]))    

            
       self.len_cumsum = np.cumsum(self.len_files)     
       print(self.data[0].shape)
       print(self.labels[0].shape)



    def __len__(self):
        result =  sum(self.len_files) - 10
        return result

    def get_index_record(self, item):
      for i_file in range(len(self.len_cumsum)):
         #print(item, self.len_cumsum[i_file])
         if (item > self.len_cumsum[i_file]):
            continue
         else:
            break
      if i_file == 0:
         index_in_file = item
      else:
         index_in_file = item  - self.len_cumsum[i_file - 1]
      nvideo = index_in_file//(self.len_record)# * LEN_RECORD_IN_SECONDS *  NTIMES_IN_SEC)
      nsec = (index_in_file - nvideo * self.len_record) # *   NTIMES_IN_SEC)

      return i_file, index_in_file, nvideo, nsec

    def __getitem__(self, item):
      sample = {}
      #print(item)
      i_file, index_in_file, nvideo, nsec = self.get_index_record(item)
      #print(i_file, nvideo, nsec )
      sample['data'] = np.zeros((HCANALS, WCANALS, NTIMES_IN_SAMPLE))
      sample_from_one_canals = []
      for i_canal in range(NCANALS):
        sample_from_one_canal = self.data[i_file][nvideo, i_canal, nsec * 128 : nsec * 128 + 128]
        sample_from_one_canals.append(sample_from_one_canal)
        #print(sample_from_one_canal.shape)
      sample_from_one_canals = np.asarray(sample_from_one_canals).copy()
      sample_from_one_canals = scipy.stats.zscore(sample_from_one_canals, axis = 0)
      for i_canal in range(NCANALS):
          sample['data'][electrode_matrix[list_electrodes[i_canal]][0],  electrode_matrix[list_electrodes[i_canal]][1]] = sample_from_one_canals[i_canal]
      #for i in range(NTIMES_IN_SAMPLE): 
      #    sample['data'][:, :, i] = scipy.stats.zscore(sample['data'][:, :, i])
      sample['data'] = torch.FloatTensor(sample['data'])
      sample['labels']  = torch.LongTensor(self.labels[i_file][nvideo])
      
      return sample

In [ ]:
def get_model():
  model = EmotionNet(HCANALS, WCANALS, NFEATURES, NTIMES_IN_SAMPLE).to(device)
  return model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model()
# def init_weights(m):
#     for name, param in m.named_parameters():
#         if 'weight' in name:
#             nn.init.normal_(param.data, mean=0, std=0.01)
#         else:
#             nn.init.constant_(param.data, 0)
            
# model.apply(init_weights)

def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

EmotionNet(
  (convs): ModuleList(
    (0): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
     

In [ ]:
files = glob.glob(os.path.join(data_dir, "*.dat"))
files.sort()
files = np.asarray(files)
nfiles = len(files)
koeff1 = 0.99
# koeff2 = 0.05
ind_train = random.sample(range(nfiles), int(nfiles * koeff1))
ind_val = list(set(range(nfiles)) - set(ind_train))
#ind_val = random.sample(ind, int(len(ind) * koeff2))
# ind_test = list(set(ind) - set(ind_val))

In [ ]:
# def normalize_data():
#   for sub in range(32):
#     for nvideo in range(40):
#         for ncanal in range(NCANALS):
#              #std = np.std(data[sub][nvideo, ncanal, :])
#              #data[sub][nvideo, ncanal, :] = np.clip(data[sub][nvideo, ncanal, :], -3 * std, 3 * std)
#              scaler = MinMaxScaler(feature_range=(-1, 1))
#              data[sub][nvideo, ncanal, :] = scaler.fit_transform(data[sub][nvideo, ncanal, :].reshape(-1, 1)).reshape(1, -1)

# normalize_data()

In [ ]:
print(ind_val)

[26]


In [ ]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 100
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="runs/segmentation_baseline"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()    

In [ ]:
type_emotion = 0

In [ ]:
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import f1_score, accuracy_score
k  = 5
labels_bin = []
for i in range(32):
  temp = labels[i] > 4.5
  #print(labels[i])
  #print(temp)
  labels_bin.append(temp)
X = np.arange(40)
y = np.array(labels_bin[0][:, type_emotion])
skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
balanced_split = skf.split(X, y)
for ind_train, ind_test in  balanced_split:
    print(ind_train, ind_test)
    print(sum(labels_bin[0][ind_train, type_emotion]))
    print(sum(labels_bin[0][ind_test, type_emotion]))
    break

[ 1  2  3  5  6  7  8  9 10 12 13 14 15 16 18 20 21 22 23 24 25 26 28 30
 31 32 33 35 36 37 38 39] [ 0  4 11 17 19 27 29 34]
16
4


In [ ]:

# train_dataset = EmotionDataset(args.data_dir, 'train', ind_train, data, labels)

# class_weights_all = [1/train_dataset.cnt[0][i] for i in range(2)]
# weights_samples =  [0] * train_dataset.__len__()
# for i in range(train_dataset.__len__()):
#     i_file, index_in_file, nvideo, nsec = train_dataset.get_index_record(i)
#     #print(train_dataset.labels[i_file][nvideo])
#     weights_samples[i] = class_weights_all[int(train_dataset.labels[i_file][nvideo, 0])]

# weighted_sampler = WeightedRandomSampler(
#     weights=weights_samples,
#     num_samples=len(weights_samples),
#     replacement=True
# )
# train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
#                               pin_memory=True, shuffle=False, drop_last=True, sampler=weighted_sampler)


# val_dataset = EmotionDataset(args.data_dir, 'val', ind_test, data, labels)
# val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
#                               pin_memory=True, shuffle=False, drop_last=False)

In [ ]:
# train_dataset = EmotionDataset(files[ind_train])
# train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
#                               pin_memory=True, shuffle=True, drop_last=True)


# val_dataset = EmotionDataset(files[ind_val])
# val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
#                               pin_memory=True, shuffle=False, drop_last=False)

In [ ]:
criterion = nn.CrossEntropyLoss(reduce = 'mean')#torch.nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
optimizer = optim.Adam(model.parameters(), lr=3e-5)#, momentum = 0.9)#, weight_decay=args.weight_decay)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
# print(train_dataset.cnt)
# print(val_dataset.cnt)
# print(files[ind_train])
# print(files[ind_val])

In [ ]:
def train(model, loader, criterion, optimizer, device, type_emotion, batch = None):
    model.train()
    train_loss = []
    inputs = []
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    #for batch in tqdm(loader, total=len(loader), desc="training...", position=0 , leave = True):
    for i, batch in enumerate(loader):

            optimizer.zero_grad()
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, type_emotion]

            #print(batch)
            #print(trg.shape)
            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            #usual cross entropy
            #output = levels_pred[:, 1:].reshape(-1, levels_pred.shape[-1])
            #trg1 = trg[:, 1:].reshape(-1)
            loss = criterion(levels_pred, trg) 

            #print("after")
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            #break
    return np.mean(train_loss)#, mid_outputs


In [ ]:
def evaluate(model, loader, criterion, device, type_emotion):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        #for s, batch in enumerate(tqdm(loader, total=len(loader), desc="validating...", position=0 , leave = True)):
        for s, batch in enumerate(loader):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, type_emotion]



            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            
            loss = criterion(levels_pred, trg) 

            epoch_loss += loss.item() 
        
    return epoch_loss / s

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

def calculate_predictions(model, loader, type_emotion):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        for i, batch in enumerate(loader):#, total=len(loader), desc="predicting...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, type_emotion]
            #print(trg)
         

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            trg_pred = levels_pred.argmax(1)
            #print(levels_pred.shape)
            
            real.extend(trg)
            pred.extend(trg_pred) 

        #print(real)
            
        #print(accuracy_score(real, pred)) 
        #print(confusion_matrix(real, pred))  
        #print(classification_report(real, pred))
        f1 = f1_score(real, pred, average = "macro")
        return (accuracy_score(real, pred)) , f1
        #plt.hist(real)

In [ ]:
def train_loop(description, type_emotion, n_epochs = 10):
      args.epochs = n_epochs
      #criterion =  fnn.mse_loss
      train_loss_min = 10000
      val_loss_min = 10000
      acc_max = -10000
      #batch = next(iter(train_dataloader))
      for epoch in range(args.epochs):
          #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
          
          train_loss = train(model, train_dataloader, criterion, optimizer ,device, type_emotion)
          #if epoch % 500 == 0:
          print(train_loss)

          if (train_loss < train_loss_min):
              train_loss_min      = train_loss
              torch.save({
                              'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                            },
                            os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", "train_cnn2.tgz")
                  )  

          val_loss = evaluate(model, val_dataloader, criterion, device, type_emotion)
          # #break
          print(val_loss)
          #break

          acc, f1 = calculate_predictions(model, val_dataloader, type_emotion)
          print(acc, f1)
          if (acc > acc_max):
              acc_max   = acc
              torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"val_{description}.tgz"))

#Результаты на датасете без лика

##Valence

In [ ]:
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd

#def get_mertics_per_subject():
k  = 5
type_emotion = 0
acc_all = []
f1_all = []

for sub in range(0, 4):
    print(f"*******{sub}*********")

    args.batch_size = 100
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    

    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    acc_sub = []
    f1_sub = []
   
    for fold,  (ind_train, ind_test) in  enumerate(balanced_split):
        inds_train = []
        inds_test = []
        inds_train.append(ind_train)
        inds_test.append(ind_test)
        print(f"fold = {fold}")
       
        train_dataset = EmotionDataset(args.data_dir, 'train', ind_train, data[sub : sub + 1], labels[sub : sub + 1])

#     class_weights_all = [1/train_dataset.cnt[0][i] for i in range(2)]
# weights_samples =  [0] * train_dataset.__len__()
# for i in range(train_dataset.__len__()):
#     i_file, index_in_file, nvideo, nsec = train_dataset.get_index_record(i)
#     #print(train_dataset.labels[i_file][nvideo])
#     weights_samples[i] = class_weights_all[int(train_dataset.labels[i_file][nvideo, 0])]

# weighted_sampler = WeightedRandomSampler(
#     weights=weights_samples,
#     num_samples=len(weights_samples),
#     replacement=True
# )
    #train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
    #                          pin_memory=True, shuffle=False, drop_last=True, sampler=weighted_sampler)

        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=True)
 
        val_dataset = EmotionDataset(args.data_dir, 'val', ind_test, data[sub : sub + 1], labels[sub : sub + 1])
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)
        device  = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = get_model()
        model.apply(initialize_weights)
        criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        optimizer = optim.Adam(model.parameters(), lr=3e-5, weight_decay=0.01)
        description = f'val_cnn2_noleak_result_{sub}_{fold}_relu'
        train_loop(description, 0, 10)
        model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"val_{description}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        acc, f1 = calculate_predictions(model, val_dataloader, type_emotion)
        #print(f"f1 = {f1} acc = {acc}")
        acc_sub.append(acc)
        f1_sub.append(f1)
        #print(acc, f1)
    acc_all.append(acc_sub)     
    f1_all.append(f1_sub)     
    pd.DataFrame(f1_all).to_csv("f1_val_cnn2_noleak_result2.csv")
    pd.DataFrame(acc_all).to_csv("acc_val_cnn2_noleak_result2.csv")

*******0*********
fold = 0
(32, 40, 7680)
(32, 4)
(8, 40, 7680)
(8, 4)
2.9631445396103357
0.9748820886015892
0.6226012793176973 0.6175309267663387
0.3804452285954827
3.8775350004434586
0.47974413646055436 0.47962821492233254
0.21482605398877672
4.44195719063282
0.509594882729211 0.5095748163235616
0.08347702281255472
4.534787341952324
0.5223880597014925 0.5223685167672947
0.042446476260298176
4.560009136795998
0.5287846481876333 0.5287846481876332
0.021419136686936804
4.476233780384064
0.5245202558635395 0.5243818696935383
0.010501139209066567
4.36704383790493
0.5266524520255863 0.5265469824293354
0.006200517441979365
4.267477065324783
0.5309168443496801 0.5308123249299721
0.004771270593138118
4.193490669131279
0.5287846481876333 0.5287075139252018
0.0039825493126715485
4.1364023089408875
0.5394456289978679 0.5393430099312453
fold = 1
(32, 40, 7680)
(32, 4)
(8, 40, 7680)
(8, 4)
3.0964468607777045
1.385007306933403
0.4669509594882729 0.4624321449530516
0.4791931311943029
2.3438502997159

In [ ]:
pd.DataFrame(f1_all).to_csv("f1_val_cnn2_noleak_result.csv")
pd.DataFrame(acc_all).to_csv("acc_val_cnn2_noleak_result.csv")

In [ ]:
pd.DataFrame(f1_all).to_csv("f1_val_cnn2_noleak_result2.csv")
pd.DataFrame(acc_all).to_csv("acc_val_cnn2_noleak_result2.csv")

##Arousal

In [ ]:
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd

#def get_mertics_per_subject():
k  = 5
type_emotion = 1
acc_all = []
f1_all = []

for sub in range(26, 32):
    print(f"*******{sub}*********")

    args.batch_size = 100
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    

    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    acc_sub = []
    f1_sub = []
   
    for fold,  (ind_train, ind_test) in  enumerate(balanced_split):
        inds_train = []
        inds_test = []
        inds_train.append(ind_train)
        inds_test.append(ind_test)
        print(f"fold = {fold}")
       
        train_dataset = EmotionDataset(args.data_dir, 'train', ind_train, data[sub : sub + 1], labels[sub : sub + 1])

#     class_weights_all = [1/train_dataset.cnt[0][i] for i in range(2)]
# weights_samples =  [0] * train_dataset.__len__()
# for i in range(train_dataset.__len__()):
#     i_file, index_in_file, nvideo, nsec = train_dataset.get_index_record(i)
#     #print(train_dataset.labels[i_file][nvideo])
#     weights_samples[i] = class_weights_all[int(train_dataset.labels[i_file][nvideo, 0])]

# weighted_sampler = WeightedRandomSampler(
#     weights=weights_samples,
#     num_samples=len(weights_samples),
#     replacement=True
# )
    #train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
    #                          pin_memory=True, shuffle=False, drop_last=True, sampler=weighted_sampler)

        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=True)
 
        val_dataset = EmotionDataset(args.data_dir, 'val', ind_test, data[sub : sub + 1], labels[sub : sub + 1])
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)
        device  = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = get_model()
        model.apply(initialize_weights)
        criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        optimizer = optim.Adam(model.parameters(), lr=3e-5, weight_decay=0.01)
        description = f'arousal_cnn2_noleak'
        train_loop(description, 1, 10)
        model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"val_{description}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        acc, f1 = calculate_predictions(model, val_dataloader, type_emotion)
        #print(f"f1 = {f1} acc = {acc}")
        acc_sub.append(acc)
        f1_sub.append(f1)
        #print(acc, f1)
    acc_all.append(acc_sub)     
    f1_all.append(f1_sub)     
    pd.DataFrame(f1_all).to_csv("f1_arousal_cnn2_noleak1.csv")
    pd.DataFrame(acc_all).to_csv("acc_arousal_cnn2_noleak1.csv")

*******26*********
fold = 0
(32, 40, 7680)
(32, 4)
(8, 40, 7680)
(8, 4)
4.240310789723146
2.7105812840163708
0.39232409381663114 0.3841599668271557
2.1409063433345996
4.552056014537811
0.37100213219616207 0.36317356863070005
1.3158138025748103
7.7657740116119385
0.3752665245202559 0.3601875433589258
0.6485314812315138
7.477760910987854
0.3603411513859275 0.3485869847031372
0.33773029321118403
7.198613882064819
0.3837953091684435 0.3735423666927034
0.1577831795929294
5.836229205131531
0.4072494669509595 0.40312213880241715
0.10106765478849411
5.693019211292267
0.3880597014925373 0.38102464349929416
0.07071119380232535
5.417939484119415
0.39232409381663114 0.3841599668271557
0.050307512038240305
5.143759846687317
0.39445628997867804 0.3860124645056606
0.03744197886829313
5.0069350600242615
0.3880597014925373 0.38044343795596924
fold = 1
(32, 40, 7680)
(32, 4)
(8, 40, 7680)
(8, 4)
5.561457991600037
2.5289024114608765
0.4200426439232409 0.42001891321742924
2.345058897608205
3.2945269048213

#LEAK

##Valence

In [ ]:
import pandas as pd
class EmotionDataset_leak(Dataset):
    def __init__ (self, data, labels_bin, transforms, indexes, interval = 1):  #indexes - индексы видео которые вошли в датасет, data - данные labels - метки бинарные
       self.data_samples = []
       self.labels = []
       self.transforms = transforms
       self.cnt = [Counter(), Counter(), Counter(),Counter()]
       for sub in range(len(data)): #sub   - человек
          for nvideo in range(NVIDEOS):
             
             for nsec in range(LEN_RECORD_IN_SECONDS):
                    self.data_samples.append(data[sub][nvideo, :32,  (3 + nsec) * NTIMES_IN_SEC : (3 + nsec + interval) * NTIMES_IN_SEC])
                    self.labels.append(labels_bin[sub][nvideo, :])
                    

       self.data_samples = np.array(self.data_samples)[np.array(indexes)]
       self.labels = np.array(self.labels)[np.array(indexes)]
       for i in range(4):
          self.cnt[i].update(self.labels[:, i])
       

    def __len__(self):
       result =  len(self.data_samples)
       return result

    def __getitem__(self, item):
      sample = {}   
      sample = {"labels": self.labels[item],
                 "data": self.data_samples[item]
      }

      data_sample = sample['data']
      #print(data_sample.shape)
      sample['data'] = np.zeros((HCANALS, WCANALS, NTIMES_IN_SAMPLE))
      #sample_from_one_canals = []
      # for i_canal in range(NCANALS):
      #   sample_from_one_canal = data_sample[nvideo, i_canal, nsec * 128 : nsec * 128 + 128]
      #   sample_from_one_canals.append(sample_from_one_canal)
      #   #print(sample_from_one_canal.shape)
      # sample_from_one_canals = np.asarray(sample_from_one_canals).copy()
      sample_from_one_canals = scipy.stats.zscore(data_sample, axis = 0)
      for i_canal in range(NCANALS):
          sample['data'][electrode_matrix[list_electrodes[i_canal]][0],  electrode_matrix[list_electrodes[i_canal]][1]] = sample_from_one_canals[i_canal]
      #for i in range(NTIMES_IN_SAMPLE): 
      #    sample['data'][:, :, i] = scipy.stats.zscore(sample['data'][:, :, i])
      sample['data'] = torch.FloatTensor(sample['data'])
      sample['labels'] = torch.LongTensor(sample['labels'])
      #sample['labels']  = torch.LongTensor(self.labels[i_file][nvideo])
      
      #  if self.transforms is not None:
      #      for t in self.transforms:
      #           sample = t(sample)
      #print(sample)         
      
      return sample

In [ ]:

from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import f1_score, accuracy_score


#def get_mertics_per_subject():
k  = 5
type_emotion = 0

acc_all = []
f1_all = []
for sub in range(0,32):
    print(f"*******{sub}*********")

    args.batch_size = 100
    indexes = np.arange(NVIDEOS * 1 * LEN_RECORD_IN_SECONDS)
    n = len(indexes)
    #X = np.arange(40)
    y = []
    for nvideo in range(NVIDEOS):
        y.extend(60 * [labels_bin[sub][nvideo, type_emotion]])
    

    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(indexes, y)
    acc_sub = []
    f1_sub = []
    for fold,  (inds_train, inds_test) in  enumerate(balanced_split):
        print(f"fold = {fold}")
        #print(inds_train, inds_test)
        #print(sum(labels_bin[sub][inds_train, type_emotion]))
        #print(sum(labels_bin[sub][inds_test, type_emotion]))
        args.batch_size = 100
        transforms = []
        #transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
        #transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
        #transforms = [to_head_matrix(),ToTensor()] 
        #transforms = [RandomAugmentation([add_noise(), None], 0.5), to_head_matrix(),ToTensor()]   
        #transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   

        train_dataset = EmotionDataset_leak(data[sub : sub + 1], labels_bin[sub : sub + 1], transforms, inds_train)
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                    pin_memory=True, shuffle=True, drop_last=True)

        val_dataset = EmotionDataset_leak(data[sub: sub + 1], labels_bin[sub  : sub + 1], transforms,  inds_test)
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)
        device  = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = get_model()
        model.apply(initialize_weights)
        criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=3e-4, momentum = 0.9)#, weight_decay=args.weight_decay)
        #optimizer = optim.Adam(model.parameters(), lr=3e-6)#, momentum = 0.9)#, weight_decay=args.weight_decay)
        
        description = f'cnn2_leak_{sub}_{fold}_subject'
        train_loop(description , 0, 3)
        model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"val_{description}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        acc, f1 = calculate_predictions(model, val_dataloader, type_emotion)
        print(f"f1 = {f1} acc = {acc}")
        acc_sub.append(acc)
        f1_sub.append(f1)
        print(acc, f1)
        #break
    acc_all.append(acc_sub)     
    f1_all.append(f1_sub)     
    pd.DataFrame(f1_all).to_csv("f1_data_val_cnn2_leak_sgd.csv")
    pd.DataFrame(acc_all).to_csv("acc_data_val_cnn2_leak_sgd.csv")

In [ ]:
pd.DataFrame(f1_all).to_csv("f1_arousal_cnn2_leak_sgd2.csv")
pd.DataFrame(acc_all).to_csv("acc_arousal_cnn2_leak_sgd2.csv")

In [ ]:
print(len(f1_all))

32


In [ ]:

from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import f1_score, accuracy_score


#def get_mertics_per_subject():
k  = 5
type_emotion = 1

acc_all = []
f1_all = []
for sub in range(0,32):
    print(f"*******{sub}*********")

    args.batch_size = 100
    indexes = np.arange(NVIDEOS * 1 * LEN_RECORD_IN_SECONDS)
    n = len(indexes)
    #X = np.arange(40)
    y = []
    for nvideo in range(NVIDEOS):
        y.extend(60 * [labels_bin[sub][nvideo, type_emotion]])
    

    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(indexes, y)
    acc_sub = []
    f1_sub = []
    for fold,  (inds_train, inds_test) in  enumerate(balanced_split):
        print(f"fold = {fold}")
        #print(inds_train, inds_test)
        #print(sum(labels_bin[sub][inds_train, type_emotion]))
        #print(sum(labels_bin[sub][inds_test, type_emotion]))
        args.batch_size = 100
        transforms = []
        #transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
        #transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
        #transforms = [to_head_matrix(),ToTensor()] 
        #transforms = [RandomAugmentation([add_noise(), None], 0.5), to_head_matrix(),ToTensor()]   
        #transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   

        train_dataset = EmotionDataset_leak(data[sub : sub + 1], labels_bin[sub : sub + 1], transforms, inds_train)
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                    pin_memory=True, shuffle=True, drop_last=True)

        val_dataset = EmotionDataset_leak(data[sub: sub + 1], labels_bin[sub  : sub + 1], transforms,  inds_test)
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)
        device  = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = get_model()
        model.apply(initialize_weights)
        criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=3e-4, momentum = 0.9)#, weight_decay=args.weight_decay)
        #optimizer = optim.Adam(model.parameters(), lr=3e-6)#, momentum = 0.9)#, weight_decay=args.weight_decay)
        
        description = f'cnn2_leak_subject'
        train_loop(description , 1, 3)
        model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"val_{description}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        acc, f1 = calculate_predictions(model, val_dataloader, type_emotion)
        print(f"f1 = {f1} acc = {acc}")
        acc_sub.append(acc)
        f1_sub.append(f1)
        print(acc, f1)
        #break
    acc_all.append(acc_sub)     
    f1_all.append(f1_sub)     
    pd.DataFrame(f1_all).to_csv("f1_arousal_cnn2_leak_sgd.csv")
    pd.DataFrame(acc_all).to_csv("acc_arousal_cnn2_leak_sgd.csv")

*******0*********
fold = 0
0.5659010653433046
0.3619055636227131
[[170  10]
 [ 22 278]]
0.9333333333333333 0.9297783629580865
0.12273557935106127
0.1372770955786109
[[171   9]
 [  6 294]]
0.96875 0.9665537856933819
0.04006049782037735
0.1267004543915391
[[172   8]
 [  9 291]]
0.9645833333333333 0.9622639764334834
[[171   9]
 [  6 294]]
f1 = 0.9665537856933819 acc = 0.96875
0.96875 0.9665537856933819
fold = 1
0.5886757875743666
0.40784429013729095
[[168  12]
 [ 32 268]]
0.9083333333333333 0.9041742286751362
0.11588066894757121
0.1756174210458994
[[173   7]
 [ 14 286]]
0.95625 0.9536830690480675
0.05403874843920532
0.1635468527674675
[[175   5]
 [ 14 286]]
0.9604166666666667 0.9581802924628231
[[175   5]
 [ 14 286]]
f1 = 0.9581802924628231 acc = 0.9604166666666667
0.9604166666666667 0.9581802924628231
fold = 2
0.5593963500700498
0.4174707420170307
[[135  45]
 [ 20 280]]
0.8645833333333334 0.8509850746268657
0.1184415758440369
0.21906737983226776
[[161  19]
 [ 16 284]]
0.9270833333333334 

In [ ]:
print(acc_all)

[[0.4904051172707889, 0.5842217484008528, 0.6439232409381663, 0.5756929637526652, 0.5756929637526652]]
